#1. Configurações de acesso ao Postgres

In [1]:
!pip install psycopg2 psycopg2-binary

     |████████████████████████████████| 3.0MB 4.2MB/s 


In [2]:
import psycopg2
class Postgres():
  def __init__(self,database='postgres',user='DataWarriorsAdmin@data-warriors-postgresql',password='DataWarriorsPassword!',host='data-warriors-postgresql.postgres.database.azure.com',port='5432'):
    self.connection = psycopg2.connect(host=host, database=database,user=user, password=password,port=port, sslmode='require')
    self.new_cursor()
 
  def get_connection(self):
    return self.connection
 
  def new_cursor(self):
    self.cursor = self.connection.cursor()
    return self.cursor
 
  def commit(self):
    self.connection.commit()
 
  def rollback(self):
    self.connection.rollback()
 
  def fetchall(self):
    return self.cursor.fetchall()
 
  def execute(self,query):
    self.cursor.execute(query)
    self.commit()
 
  def search(self,query):
    self.new_cursor()
    self.cursor.execute(query)
    self.commit()
    return self.fetchall()
 
  def insertMany(self,query,data_list):
    try:
      self.new_cursor()
      self.cursor.executemany(query, data_list)
      self.commit()    
      print('Execução da transação conclída com sucesso:', query)
    except Exception as error:
      self.rollback()
      print('Transação falhou:',error)
    
  def closeConnection(self):
    self.connection.close()

In [3]:
p = Postgres()

#2. Criação do Menu
Funcionalidades:
  1. Panorama diário de quantidade de casos confirmados de COVID-19 dos 10 países do mundo com maiores números.
  2. Panorama diário de quantidade de mortes de COVID-19 dos 10 países do mundo com números.
  3. Total de mortes por COVID-19 dos 10 países do mundo com maiores números.
  4. Total de casos confirmados por COVID-19 dos 10 países do mundo com maiores números.

In [6]:
import pandas as pd

###2.1 Definindo a classe filho Menu que tem como pai a classe Postgres

In [7]:
class Menu(Postgres):   
 
  def dailyCases(self):
    print('\nNúmero de casos confirmados de Covid-19 registrados na data mais recente:\n')
    return display(pd.DataFrame(super().search('''select c.countryname as Pais ,
                               s.newconfirmed as Numero_Diario_Casos, 
                               s.lastupdated as Data_Atualizacao from summary as s
                               inner join country as c
                               on s.countrycode = c.countrycode 
                               order by s.newconfirmed desc 
                               limit 10;'''),columns=['País','Casos Diários','Data de Atualização']))
 
  def dailyDeaths(self):
    print('\nNúmero de mortes por Covid-19 registradas na data mais recente:\n ')
    return display(pd.DataFrame(super().search('''select  c.countryname as Pais,
                               s.newdeaths as Numero_de_mortes_diario,
                               s.lastupdated as Data_de_atualizacao
                               from summary as s 
                               inner join country as c 
                               on s.countrycode = c.countrycode 
                               order by s.newdeaths desc 
                               limit 10;'''),columns=['País','Mortes Diárias','Data de Atualização']))
 
  def totalCases(self):
    print('\nTotal acumulado de casos confirmados de Covid-19 registrados até a data mais recente:\n')
    return display(pd.DataFrame(super().search('''select c.countryname as Pais,
                              s.totalconfirmed as "Número_de_mortes_total",
                              s.lastupdated as "Data_de_atualização"
                              from summary as s 
                              inner join country as c 
                              on s.countrycode = c.countrycode
                              order by s.totalconfirmed desc
                              limit 10;'''),columns=['País','Total de Casos','Data de Atualização']))
 
  def totalDeaths(self):
    print('\nTotal acumulado de mortes por Covid-19 registradas até a data mais recente:\n ')
    return display(pd.DataFrame(super().search('''select  c.countryname as Pais,
                              s.totaldeaths as Numero_de_casos_total,
                              s.lastupdated as Data_de_atualizacao
                              from summary as s
                              inner join country as c 
                              on s.countrycode = c.countrycode
                              order by s.totaldeaths desc
                              limit 10;'''),columns=['País','Total de Mortes','Data de Atualização']))

###2.2 Definindo função de recepção do usuário

In [8]:
def recebeOpcaoUsuario():
    opcao = 0
 
    print("Digite a opção que deseja executar:\n"
          "1. Panorama diário de quantidade de casos confirmados de COVID-19 dos 10 países do mundo com mais casos.\n"
          "2. Panorama diário de quantidade de mortes de COVID-19 dos 10 países do mundo com mais mortes.\n"
          "3. Total de mortes por COVID-19 dos 10 países do mundo com mais casos.\n"
          "4. Total de casos confirmados por COVID-19 dos 10 países do mundo com maiores números.\n"
          "5. Sair.\n")
 
    while opcao < 1 or opcao > 5:
        try:
            opcao = int(input("\nDigite uma opção de 1 a 5: "))
        except:
            print("\nOpção inválida. Digite novamente\n")
 
    return opcao

###2.3 Chamando os modulos

In [9]:
if __name__ == '__main__':
    opcao = recebeOpcaoUsuario()
    instancia = Menu()
 
    while opcao >= 1 and opcao <= 5:
            if opcao == 1: 
                instancia.dailyCases()
 
            elif opcao == 2: 
                instancia.dailyDeaths()
 
            elif opcao == 3: 
                instancia.totalDeaths()
 
            elif opcao == 4: 
                instancia.totalCases()
 
            else: #sair do programa
                print("\nVocê saiu do programa. Obrigado e use sempre a máscara!\n")
                break
                
            if opcao != 5:
                opcao = recebeOpcaoUsuario()

Digite a opção que deseja executar:
1. Panorama diário de quantidade de casos confirmados de COVID-19 dos 10 países do mundo com mais casos.
2. Panorama diário de quantidade de mortes de COVID-19 dos 10 países do mundo com mais mortes.
3. Total de mortes por COVID-19 dos 10 países do mundo com mais casos.
4. Total de casos confirmados por COVID-19 dos 10 países do mundo com maiores números.
5. Sair.


Digite uma opção de 1 a 5: 1

Número de casos confirmados de Covid-19 registrados na data mais recente:



,País,Casos Diários,Data de Atualização
0,United States of America,1169754,2021-03-07
1,Ecuador,294999,2021-03-09
2,Brazil,70764,2021-03-09
3,France,23143,2021-03-09
4,Italy,19725,2021-03-09
5,India,18599,2021-03-07
6,Turkey,13755,2021-03-09
7,Sweden,11014,2021-03-09
8,Czech Republic,10524,2021-03-09
9,Poland,9953,2021-03-09


Digite a opção que deseja executar:
1. Panorama diário de quantidade de casos confirmados de COVID-19 dos 10 países do mundo com mais casos.
2. Panorama diário de quantidade de mortes de COVID-19 dos 10 países do mundo com mais mortes.
3. Total de mortes por COVID-19 dos 10 países do mundo com mais casos.
4. Total de casos confirmados por COVID-19 dos 10 países do mundo com maiores números.
5. Sair.


Digite uma opção de 1 a 5: 6

Digite uma opção de 1 a 5: 3

Total acumulado de mortes por Covid-19 registradas até a data mais recente:
 


,País,Total de Mortes,Data de Atualização
0,United States of America,515148,2021-03-07
1,Brazil,268370,2021-03-09
2,Mexico,191789,2021-03-09
3,France,177902,2021-03-09
4,India,157853,2021-03-07
5,United Kingdom,124736,2021-03-07
6,Italy,100479,2021-03-09
7,Russian Federation,88315,2021-03-09
8,Germany,72534,2021-03-09
9,Spain,71138,2021-03-07


Digite a opção que deseja executar:
1. Panorama diário de quantidade de casos confirmados de COVID-19 dos 10 países do mundo com mais casos.
2. Panorama diário de quantidade de mortes de COVID-19 dos 10 países do mundo com mais mortes.
3. Total de mortes por COVID-19 dos 10 países do mundo com mais casos.
4. Total de casos confirmados por COVID-19 dos 10 países do mundo com maiores números.
5. Sair.


Digite uma opção de 1 a 5: 5

Você saiu do programa. Obrigado e use sempre a máscara!

